In [ ]:
%load_ext autoreload
%load_ext line_profiler

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import inspect
from IPython import display

import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import kwant 

import functools as ft

import sns_system, plotting_results
import topology, spectrum, supercurrent
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

# Define system

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'L_down' :  1000,
             'L_up' : 1000,
             'Lm' : 250,
             'Lx' : 12.5,
             'a' :  12.5,
             'mu_from_bottom_of_spin_orbit_bands': True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = 0.18,
                 Delta_right = 0.18,
                 B = .5,
                 phase = 0*np.pi,
                 T = .25,
                 V = 1)

params = dict(**constants,
              **params_raw)

# Make SNS junction

In [ ]:
syst = sns_system.make_sns_system(**syst_pars)

In [ ]:
# spectrum.find_gap_of_lead(syst.leads[0], params, tol=1e-2)

In [ ]:
syst = sns_system.make_wrapped_system(**syst_pars)

In [ ]:
kf

In [ ]:
p = params.copy()
def energy(k_x):
    p.update(k_x=k_x)
    ham = syst.hamiltonian_submatrix(params=p, sparse=True)
    e = spectrum.sparse_diag(ham, 1, 0, return_eigenvectors=False)
    return abs(e[0])

ef_max = params['mu'] + params['m_eff']*params['alpha_middle']**2/params['hbar']**2 + params['g_factor_middle']*params['mu_B']*params['B']
kf = np.sqrt(ef_max*2*params['m_eff'])/params['hbar']*np.pi/2*syst_pars['a']
kmax = min(kf, np.pi)

learner = adaptive.Learner1D(energy, [0, kmax])

In [ ]:
# runner = adaptive.runner.simple(learner, goal = lambda l: l.loss() < 1e-2)
runner = adaptive.Runner(learner, goal = lambda l: l.loss() < 1e-2)

runner.live_info()


In [ ]:
learner.loss()

In [ ]:
learner.plot()

In [ ]:
%%time
syst = sns_system.make_sns_system(**syst_pars)
spectrum.gap_minimizer(syst.leads[0], params, 0.05)

In [ ]:
%%time
supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst)